<a href="https://colab.research.google.com/github/EtienneGomez/Sentiment_Analysis/blob/main/Practica5Sentiment_AnalysisMod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import os, pickle
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.svm import LinearSVC
import pandas as pd
import spacy
import os, pickle


In [ ]:
import pandas as pd

# Cargamos el archivo excel
excel_path = 'Rest_Mex_2022.xlsx'
df = pd.read_excel(excel_path)
df.head()


,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel


In [ ]:
import pandas as pd
import os, pickle
!python -m spacy download es_core_news_sm

nlp = spacy.load('es_core_news_sm')

# Función para lematizar el texto
def lematizar_texto(texto):
    # Convertir la entrada a cadena si no lo es
    texto = str(texto) if not isinstance(texto, str) else texto
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc])

# Cargar tus datos
df = pd.read_excel('Rest_Mex_2022.xlsx')

# Lematización a la columna 'Opinion'
df['Opinion_Lematizada'] = df['Opinion'].apply(lematizar_texto)

# DataFrame solo con las columnas Opinión y Polaridad
df_reducido = df[['Opinion_Lematizada', 'Polarity']]

# Guardar el nuevo DataFrame en un archivo pickle
df_reducido.to_pickle('lematizado_polaridad.pkl')


2023-11-25 01:00:37.223866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 01:00:37.223949: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 01:00:37.223997: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 01:00:40.552790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 47.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
df_reducido = pd.read_pickle('lematizado_polaridad.pkl')
df_reducido.head()


,Opinion_Lematizada,Polarity
0,"piensir dos vez antes de ir a este hotel , tú ...",1
1,cuatro de yo ir recientemente a Eddie's Place ...,1
2,seguir corto y simple : limpieza \n - bad . te...,1
3,al reservar uno hotel con multipropiedad Mayan...,1
4,"no perder su tiempo ni dinero , venir porque t...",1


In [ ]:
df_reducido.to_csv('noticiaPolaridad.csv', index=False)

In [ ]:
df = pd.read_csv('noticiaPolaridad.csv')
df.head()

,Opinion_Lematizada,Polarity
0,"piensir dos vez antes de ir a este hotel , tú ...",1
1,cuatro de yo ir recientemente a Eddie's Place ...,1
2,seguir corto y simple : limpieza \n - bad . te...,1
3,al reservar uno hotel con multipropiedad Mayan...,1
4,"no perder su tiempo ni dinero , venir porque t...",1


In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('noticiaPolaridad.csv')

# 'Opinion_Lematizada' es el texto y 'Polarity' es la etiqueta
X = df['Opinion_Lematizada']
y = df['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

18594    también ahora , abierto solo con comida para l...
19690    desde que llegar al hotel el personal a estado...
26183    excelente servicio y poner el súper tazón en t...
26029    excelente el desayuno mucho abundante gran var...
16566    tener uno masaje de 80 minuto por fin y pareja...
Name: Opinion_Lematizada, dtype: object
18594    5
19690    5
26183    5
26029    5
16566    5
Name: Polarity, dtype: int64
9730     el cena ser fenomenal , y como siempre , el se...
7442     yo encaminar a Puerto Vallarta por primero vez...
25844    gran lugar para comer ! Camarón azul , pargo ,...
9565     buen comida , abundante , precio regular , sol...
718      cenar dos vez en el período de dos semana 5 de...
Name: Opinion_Lematizada, dtype: object
9730     5
7442     4
25844    5
9565     5
718      2
Name: Polarity, dtype: int64


In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names[2950:2960])

['antipasto' 'antipático' 'antiquísimasny' 'antisarro' 'anto' 'antogir'
 'antojar' 'antoje' 'antojito' 'antojitos']


In [ ]:
print(X_train_tfidf.shape)
print(X_train_tfidf.toarray()[2950:2960, 2950:2960])


(24169, 36643)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
primer_documento = X_train.iloc[0]
print("Texto del primer documento:", primer_documento)


Texto del primer documento: también ahora , abierto solo con comida para llevar y a domicilio ser el mejór de él mejór : mismo sabor , comida caliente y servicio amable ... como siempre ! ! ! 
 en mi servicio para llevar , como prometido , yo respetar el 10% de descuento ! 
 Con Sabrosa Italia valer el pena quedar él en casa ! 
 gracias y esperar que todo pasar mucho pronto , tener gana de visitar su hermoso lugar , uno verdadero rincón de Italia en México !


In [ ]:
doc_id = 0  # Índice del primer documento en X_train
feature_names = vectorizer.get_feature_names_out()
doc_tfidf = X_train_tfidf[doc_id].toarray().flatten()
tfidf_pairs = zip(feature_names, doc_tfidf)
print(sorted(tfidf_pairs, key=lambda x: x[1], reverse=True)[:10])  # Imprime las 10 características con mayor TF-IDF


[('mejór', 0.5263916890154461), ('italia', 0.38009278590692014), ('domicilio', 0.20903510697563024), ('sabrosa', 0.19828217440622087), ('prometido', 0.19640665181850048), ('rincón', 0.18737993182617102), ('llevar', 0.18160624141057605), ('respetar', 0.16983778142083716), ('gana', 0.14069912094095566), ('descuento', 0.14030212498246497)]


In [ ]:
# Modelo LinealSVC + TF-IDF
clf_polarity = LinearSVC(  max_iter=30000, C=1.0, verbose=1)
clf_polarity.fit(X_train_tfidf, y_train)

# conjunto de prueba
y_pred = clf_polarity.predict(X_test_tfidf)

# predicciones
print(y_pred)

# precisión
print("Accuracy:", accuracy_score(y_test, y_pred))

#  matriz de confusión
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#  informe de clasificación
print("Classification Report:\n", classification_report(y_test, y_pred))

# Realizamos validación cruzada
print('Realizando validación cruzada...')
cv_results = cross_validate(clf_polarity, X_train_tfidf, y_train, cv=5, scoring='f1_macro', verbose=3, n_jobs=-1)

# Imprimimos los resultados de la validación cruzada
print(cv_results)

# Promedio de f1_macro en validación cruzada
print('Media de f1_macro en validación cruzada: ', np.mean(cv_results['test_score']))

[LibLinear][5 5 5 ... 5 5 5]
Accuracy: 0.7360582492139666
Confusion Matrix:
 [[  42   12   14   13   23]
 [  15   16   44   37   33]
 [  15   18  127  135  127]
 [   4    5   58  397  699]
 [   3    1   41  298 3866]]
Classification Report:
               precision    recall  f1-score   support

           1       0.53      0.40      0.46       104
           2       0.31      0.11      0.16       145
           3       0.45      0.30      0.36       422
           4       0.45      0.34      0.39      1163
           5       0.81      0.92      0.86      4209

    accuracy                           0.74      6043
   macro avg       0.51      0.42      0.45      6043
weighted avg       0.70      0.74      0.71      6043

Realizando validación cruzada...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'fit_time': array([4.14559937, 4.04977679, 2.54048061, 2.54341555, 1.71117449]), 'score_time': array([0.01301503, 0.01199722, 0.01125789, 0.01189423, 0.00687099]), 'test_score': array([0.42312482, 0.42208687, 0.43041926, 0.40856182, 0.41911431])}
Media de f1_macro en validación cruzada:  0.4206614135168924


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.5s finished


In [ ]:
# Modelo REGRESIÓN LOGÍSITCA + TF-IDF
clf_polarity = LogisticRegression(max_iter=30000, C=1.0, verbose=1)
clf_polarity.fit(X_train_tfidf, y_train)

y_pred = clf_polarity.predict(X_test_tfidf)

print(y_pred)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("Classification Report:\n", classification_report(y_test, y_pred))

print('Realizando validación cruzada...')
cv_results = cross_validate(clf_polarity, X_train_tfidf, y_train, cv=5, scoring='f1_macro', verbose=3, n_jobs=-1)

print(cv_results)
print('Media de f1_macro en validación cruzada: ', np.mean(cv_results['test_score']))

[5 5 5 ... 5 5 5]
Accuracy: 0.740857190137349
Confusion Matrix:
 [[  23   11   21   12   37]
 [  10    8   51   36   40]
 [  11    6  117  142  146]
 [   2    0   42  360  759]
 [   0    0   25  215 3969]]
Classification Report:
               precision    recall  f1-score   support

           1       0.50      0.22      0.31       104
           2       0.32      0.06      0.09       145
           3       0.46      0.28      0.35       422
           4       0.47      0.31      0.37      1163
           5       0.80      0.94      0.87      4209

    accuracy                           0.74      6043
   macro avg       0.51      0.36      0.40      6043
weighted avg       0.70      0.74      0.71      6043

Realizando validación cruzada...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'fit_time': array([17.61185217, 19.16153026, 15.38733268, 16.61011577,  9.7882843 ]), 'score_time': array([0.01397109, 0.01377177, 0.0219419 , 0.01170039, 0.00766516]), 'test_score': array([0.37558498, 0.37655793, 0.38240725, 0.37030038, 0.36378017])}
Media de f1_macro en validación cruzada:  0.37372614209236676


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.9s finished


In [ ]:
from sklearn.neural_network import MLPClassifier
clf_polarity = MLPClassifier(hidden_layer_sizes=(100,), max_iter = 1000, random_state=1)
clf_polarity.fit(X_train_tfidf, y_train)

y_pred = clf_polarity.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("Classification Report:\n", classification_report(y_test, y_pred))

print('Realizando validación cruzada...')
cv_results = cross_validate(clf_polarity, X_train_tfidf, y_train, cv=5, scoring='f1_macro', verbose=3, n_jobs=-1)

print(cv_results)
print('Media de f1_macro en validación cruzada: ', np.mean(cv_results['test_score']))

Accuracy: 0.7087539301671355
Confusion Matrix:
 [[  35   18   21   11   19]
 [  17   27   40   31   30]
 [  20   20  148  134  100]
 [   6    7  103  439  608]
 [   6    4   70  495 3634]]
Classification Report:
               precision    recall  f1-score   support

           1       0.42      0.34      0.37       104
           2       0.36      0.19      0.24       145
           3       0.39      0.35      0.37       422
           4       0.40      0.38      0.39      1163
           5       0.83      0.86      0.85      4209

    accuracy                           0.71      6043
   macro avg       0.48      0.42      0.44      6043
weighted avg       0.70      0.71      0.70      6043

Realizando validación cruzada...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


{'fit_time': array([1144.34514832, 1083.86048174, 1091.33300042, 1200.33347869,
        712.75078201]), 'score_time': array([0.0738163 , 0.07859659, 0.08518147, 0.11387444, 0.03328848]), 'test_score': array([0.41382076, 0.41215883, 0.43919007, 0.40570608, 0.41822304])}
Media de f1_macro en validación cruzada:  0.41781975548998657


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 48.2min finished
